# Initial selection of data 

The original Dataset is taken from 
<a href="https://www.kaggle.com/competitions/m5-forecasting-accuracy">Kaggle's M5 competition</a>. 

In this notebook we'll simply take out data for a single item FOODS_3_586. 

In [1]:
import pandas as pd

### Loading the data

Load and look at the structure of the original data.

In [2]:
df = pd.read_csv('../data/sales_train_evaluation.csv')
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


### Selecting and stacking

Select item FOODS_3_586 and stack columns.

In [3]:
dft = df[df['item_id']=='FOODS_3_586'][[c for c in df.columns if c.startswith('d_')]+['item_id', 'store_id']].set_index(['store_id', 'item_id']).stack().to_frame()
dft.index = dft.index.set_names(['store_id', 'item_id', 'd'])
dft = dft.rename(columns={0:'quantity'})
dft.head()

quantity
store_id item_id     d            
CA_1     FOODS_3_586 d_1        42
                     d_2        36
                     d_3        30
                     d_4        23
                     d_5        27

### Merge with calendar to get datetime column

In [6]:
df_cal = pd.read_csv('../data/calendar.csv')
df_cal['date'] = pd.to_datetime(df_cal['date'])
df_cal.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [7]:
df_cal.dtypes

date            datetime64[ns]
wm_yr_wk                 int64
weekday                 object
wday                     int64
month                    int64
year                     int64
d                       object
event_name_1            object
event_type_1            object
event_name_2            object
event_type_2            object
snap_CA                  int64
snap_TX                  int64
snap_WI                  int64
dtype: object

In [8]:
df_merged = pd.merge(dft.reset_index(), df_cal[['date', 'd']], how='inner', on='d').set_index(['store_id', 'item_id', 'date']).sort_index()
df_merged.head()

d  quantity
store_id item_id     date                     
CA_1     FOODS_3_586 2011-01-29  d_1        42
                     2011-01-30  d_2        36
                     2011-01-31  d_3        30
                     2011-02-01  d_4        23
                     2011-02-02  d_5        27

### Drop columns and index

Drop: 
* item_id index (not needed since we will be working with one item only)
* d column (instead of date)

In [9]:
df_final = df_merged[['quantity']].droplevel('item_id')

### Save to file

In [10]:
df_final.to_csv('../data/single_item_data.csv')